In [6]:
import sys
sys.path.append("../../")

import torch
import torchvision.utils
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import optim
from torch.utils.data import DataLoader
from torchvision import models
from torchsummary import summary

import numpy as np
import cv2
import pandas as pd
from sklearn.metrics import accuracy_score

import src.dataPreparation.CreatePartial as create_partial
import src.neuralNetworksArch.BstCnn as bst
import src.utils.Visual as vis
import src.utils.Checkpoint as ckp
import src.utils.Metrics as metrics

from src.config.Param import *

In [7]:
MODEL_PATH = [
    '../../models/PARTIAL_3 #1.pth',
    '../../models/PARTIAL_3 #2.pth',
    '../../models/PARTIAL_3 #3.pth',
    '../../models/PARTIAL_3 #4.pth'
]
PATH = '../../dataset/testing/same_cam/'
DATATEST_PATH = PATH + 'testing.csv'
IMAGES_PATH = PATH + '/images/full/'

In [8]:
def cv_image2tensor(img, convert=True):
    if convert:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img[:, :, ::-1].transpose((2, 0, 1)).copy()
    img = torch.from_numpy(img).float() / 255.0
    img = [img]
    img_tensors = torch.stack(img)

    return img_tensors

In [9]:
models = []
for index, path in enumerate(MODEL_PATH):
    data = {}
    model  = bst.BstCnn()
    checkpoint = ckp.load_checkpoint(load_dir=path)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    
    data['id'] = 'PART-' + str(index+1)
    data['model'] = model
    data['min_dist'] = checkpoint['dist'][0]
    data['max_dist'] = checkpoint['dist'][1]
    data['threshold'] = checkpoint['threshold'] + 0.1
#     data['threshold'] = checkpoint['threshold']
    data['threshold_list'] = checkpoint['threshold_list']
    models.append(data)

In [11]:
df = pd.read_csv(DATATEST_PATH)
y_true = []
y_pred = [[],[],[],[]]

verbose = False
with torch.no_grad():
    for index, data in df.iterrows():
        if verbose == True:
            print('No-{}'.format(index+1))
            print('-'*50)
        img1 = cv2.imread(IMAGES_PATH + data['image_1'])
        img2 = cv2.imread(IMAGES_PATH + data['image_2'])
        label = data['label']

        img1_part = list(create_partial.partial_image_3(img1))
        img2_part = list(create_partial.partial_image_3(img2))

        for i, (input1, input2, model) in enumerate(zip(img1_part, img2_part, models)):
            input1 = cv_image2tensor(input1, False)
            input2 = cv_image2tensor(input2, False)
            out1, out2 = model['model'](input1, input2)
            euclidean_distance = F.pairwise_distance(out1, out2)
            
            dist = metrics.normalize_data(euclidean_distance.item(), model['max_dist'])
            pred = 0.0 if dist <= model['threshold'] else 1.0
            y_pred[i].append(pred)
            
            if verbose == True:
                print('PART-{} DISTANCE => {}'.format((i+1), dist))
                print('Threshold => {}'.format(model['threshold']))
                print('-'*50)
        
        y_true.append(float(label))
        concatenated = torch.cat((cv_image2tensor(img1, False), cv_image2tensor(img2, False)),0)
        
        if verbose == True:
            vis.imshow(torchvision.utils.make_grid(concatenated))
            print('='*50, end='\n\n')

for index, pred in enumerate(y_pred):
    acc = accuracy_score(np.array(y_true), np.array(pred))
    print('Accuracy Part-{}: {}'.format(index+1, acc))

Accuracy Part-1: 0.826797385620915
Accuracy Part-2: 0.803921568627451
Accuracy Part-3: 0.7549019607843137
Accuracy Part-4: 0.7189542483660131
